In [125]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
import Packages.ClusteringHelper as ch
from Packages.TimeEvolving import DataEvolver
from textdistance import DamerauLevenshtein, Levenshtein
import numpy as np
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from collections import Counter
from Packages.TimeEvolving import Cluster

In [127]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
ents_data = data[data['entities'] != ''].copy()

In [128]:
ents_data = ch.add_entities_embedding(ents_data,
                                      "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings")
ents_data_filtered = ents_data.copy()

In [129]:
total_clusters = []
documents = set(ents_data.documents)
evolving = DataEvolver(documents, ents_data, randomly=False, step=20)
gold_entities = []
###################################################################
iterator = evolving.__iter__()
_ = iterator.__next__()
###################################################################
current_mentions = list(evolving.get_current_data().mentions)
current_encodings = list(evolving.get_current_data()['encodings'].values)
current_entities = list(evolving.get_current_data()['entities'].values)
def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return Levenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())

def dam_lev_metric(x, y):
    i, j = int(x[0]), int(y[0])  # extract indices
    if len(current_mentions[i]) < 4:
        if current_mentions[i] == current_mentions[j]:
            return 0
        else:
            return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower()) + 3
    else:
        return DamerauLevenshtein().distance(current_mentions[i].lower(), current_mentions[j].lower())

X = np.arange(len(current_mentions)).reshape(-1, 1)
clusterizator1 = DBSCAN(metric=lev_metric, eps=1, min_samples=0, n_jobs=-1)
cluster_numbers = clusterizator1.fit_predict(X)

In [130]:
cee_dict = {k: {'entities': [], 'mentions': [], 'encodings': [], 'sotto_clusters': None} for k in set(cluster_numbers)}
for i, cluster in enumerate(cluster_numbers):
    cee_dict[cluster]['entities'].append(current_entities[i])
    cee_dict[cluster]['mentions'].append(current_mentions[i])
    cee_dict[cluster]['encodings'].append(current_encodings[i])
cee_list = cee_dict.values()
# cee_list

In [131]:
clusterizator2 = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.035, linkage="single")
for cluster in cee_dict.keys():
    try:
        cee_dict[cluster]['sotto_clusters'] = clusterizator2.fit_predict(cee_dict[cluster]['encodings'])
    except ValueError:
        cee_dict[cluster]['sotto_clusters'] = np.zeros(1, dtype=np.int8)

In [132]:
sottocluster_list = []
for el in cee_list:
    sotto_cluster = {k: Cluster() for k in set(el['sotto_clusters'])}
    for i, key in enumerate(el['sotto_clusters']):
        sotto_cluster[key].add_element(mention=el['mentions'][i], entity=el['entities'][i], encodings=el['encodings'][i])
    sottocluster_list.append(sotto_cluster)

In [133]:
sottocluster_list = [clusters_dict[key] for clusters_dict in sottocluster_list for key in clusters_dict ]

In [134]:
# for x in sottocluser_list:
#     print(Counter(x.entities))

# STEP FINALE


In [135]:
current_clusters = total_clusters + sottocluster_list

In [136]:
sotto_encodings = [x.encodings_mean for x in current_clusters]

In [137]:
clusterizator3 = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.015, linkage="single")
cluster_numbers = clusterizator3.fit_predict(sotto_encodings)

In [138]:
final_clusters = {k: Cluster() for k in set(cluster_numbers)}

for i, x in enumerate(current_clusters):
    try:
        final_clusters[cluster_numbers[i]] =  final_clusters[cluster_numbers[i]] + x
    except:
        print(cluster_numbers[i], final_clusters[cluster_numbers[i]], x)

In [139]:
gold_entities = gold_entities + current_entities
total_clusters = list(final_clusters.values())

CEAFm

In [140]:
# precision
best_alignment = ch.get_optimal_alignment([x.count_ents for x in total_clusters], set(gold_entities), is_dict=False)

In [141]:
CEAFm_f1 = sum(best_alignment.values()) / len(gold_entities)
print(CEAFm_f1)


0.8664921465968587
